In [1]:
import requests
import logging
import zipfile
import io
import json
from minio import Minio

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
minio_connection = ""  

In [4]:
# carregar para funcionar
try:
    minio_conn = json.loads(minio_connection)
except json.JSONDecodeError:
    with open('../variables/minio_connection.json', "r") as minio_connection_file:
        minio_conn = json.loads(minio_connection_file.read())

In [5]:
s3_client = None

try:
    endpoint_raw = minio_conn["endpoint"]
    access_key = minio_conn["access_key"]
    secret_key = minio_conn["key"]

    endpoint_sem_http = endpoint_raw.replace("http://", "").replace("https://", "")
    is_secure = endpoint_raw.startswith("https")

    s3_client = Minio(
        endpoint=endpoint_sem_http,
        access_key=access_key,
        secret_key=secret_key,
        secure=is_secure
    )

    logging.info("Cliente MinIO criado com sucesso.")

except KeyError as e:
    logging.error(f"Erro de configuração: chave ausente - {e}")
except Exception as e:
    logging.error(f"Erro ao inicializar o cliente MinIO: {e}")

2025-08-07 17:43:31,343 - INFO - Cliente MinIO criado com sucesso.


In [6]:
url = "https://arquivos.receitafederal.gov.br/dados/cno/cno.zip"
bucket = "landing"
caminho_destino = "cno/cno/" 

In [7]:
def baixar_e_subir_zip_para_minio(url, bucket, caminho_destino):
    if not s3_client:
        logging.error("Cliente MinIO não foi inicializado.")
        return

    try:
        # 1. Baixar o arquivo .zip da URL
        logging.info(f"Baixando arquivo de {url}")
        response = requests.get(url)
        response.raise_for_status()

        zip_bytes = io.BytesIO(response.content)

        # 2. Abrir o ZIP e iterar sobre os arquivos internos
        with zipfile.ZipFile(zip_bytes) as z:
            for nome_arquivo in z.namelist():
                logging.info(f"Extraindo: {nome_arquivo}")

                with z.open(nome_arquivo) as arquivo_interno:
                    arquivo_bytes = arquivo_interno.read()

                    # Caminho final no bucket MinIO (sem alterar o nome do arquivo)
                    objeto_minio = f"{caminho_destino}{nome_arquivo}"

                    # Upload para MinIO
                    s3_client.put_object(
                        bucket_name=bucket,
                        object_name=objeto_minio,
                        data=io.BytesIO(arquivo_bytes),
                        length=len(arquivo_bytes),
                    )

                    logging.info(f"Arquivo {nome_arquivo} enviado para MinIO em {objeto_minio}")

        logging.info("Todos os arquivos do ZIP foram processados e enviados para o MinIO com sucesso.")

    except Exception as e:
        logging.error(f"Erro ao processar e enviar arquivos: {e}")
        


In [8]:
baixar_e_subir_zip_para_minio(url, bucket, caminho_destino)

2025-08-07 17:43:31,400 - INFO - Baixando arquivo de https://arquivos.receitafederal.gov.br/dados/cno/cno.zip
2025-08-07 17:46:21,762 - INFO - Extraindo: cno.csv
2025-08-07 17:46:27,766 - INFO - Arquivo cno.csv enviado para MinIO em cno/cno/cno.csv
2025-08-07 17:46:27,767 - INFO - Extraindo: cno_cnaes.csv
2025-08-07 17:46:28,532 - INFO - Arquivo cno_cnaes.csv enviado para MinIO em cno/cno/cno_cnaes.csv
2025-08-07 17:46:28,533 - INFO - Extraindo: cno_vinculos.csv
2025-08-07 17:46:28,734 - INFO - Arquivo cno_vinculos.csv enviado para MinIO em cno/cno/cno_vinculos.csv
2025-08-07 17:46:28,735 - INFO - Extraindo: cno_areas.csv
2025-08-07 17:46:30,645 - INFO - Arquivo cno_areas.csv enviado para MinIO em cno/cno/cno_areas.csv
2025-08-07 17:46:30,646 - INFO - Extraindo: cno_totais.csv
2025-08-07 17:46:30,703 - INFO - Arquivo cno_totais.csv enviado para MinIO em cno/cno/cno_totais.csv
2025-08-07 17:46:30,704 - INFO - Todos os arquivos do ZIP foram processados e enviados para o MinIO com sucesso